## Video Summarizer Using Bart

The code defines two functions, `write_srt` and `format_timestamp`. 
- The `write_srt` function takes in an iterator of dictionaries (representing transcript segments) and a file object and writes the transcript data to the file in SubRip (.srt) format. 
- The `format_timestamp` function converts a timestamp in seconds to a string in the format "HH:MM:SS,mmm" (hours, minutes, seconds, and milliseconds) and is used by the write_srt function to format the start and end timestamps of each transcript segment.

In [2]:
from datetime import timedelta
from typing import Iterator, TextIO

def write_srt(transcript: Iterator[dict], file: TextIO):
    for i, segment in enumerate(transcript, start=1):
        print(
            f"{i}\n"
            f"{format_timestamp(segment['start'], always_include_hours=True)} --> "
            f"{format_timestamp(segment['end'], always_include_hours=True)}\n"
            f"{segment['text'].strip().replace('-->', '->')}\n",
            file=file,
            flush=True,
        )

def format_timestamp(seconds: float, always_include_hours: bool = False):
    assert seconds >= 0, "non-negative timestamp expected"
    timestamp = timedelta(seconds=seconds)
    total_seconds = int(timestamp.total_seconds())
    hours, remainder = divmod(total_seconds, 3600)
    minutes, seconds = divmod(remainder, 60)
    hours_marker = f"{hours}:" if always_include_hours or hours > 0 else ""
    return f"{hours_marker}{minutes:02d}:{seconds:02d}.{timestamp.microseconds // 1000:03d}"

### Using video path, a model path, and a task (either "stt" or "asr") and generate subtitle for the video

- The code defines a function `"get_audio"` that takes a list of file paths and extracts the audio from them using FFmpeg library, it stores the audio in .wav format in a temp directory and returns a dictionary of the original file path as key and the audio file path as value.
- The function `"generate_subtitles"` takes in an audio file path, an srt file path, and a transcribe function, it transcribes the audio, generates subtitles and writes the subtitles in srt format at the provided file path.
- The function `"get_subtitles"` takes a list of file paths, a flag to indicate if the subtitles should be written to a provided directory or a temp directory, an output directory path, and a transcribe function, it calls the get_audio function to extract audio from the provided file paths, calls the generate_subtitles function to generate subtitles for each audio file, and returns a dictionary of file path as key and srt file path as value.

In [3]:
import os
import whisper
import warnings
import tempfile
import ffmpeg

def get_audio(paths):
    temp_dir = tempfile.gettempdir()
    audio_paths = {}
    for path in paths:
        filename = os.path.basename(path).split('.')[0]
        print(f"Extracting audio from {os.path.basename(path)}...")
        output_path = os.path.join(temp_dir, f"{filename}.wav")

        ffmpeg.input(path).output(
            output_path,
            acodec="pcm_s16le", ac=1, ar="16k"
        ).run(quiet=True, overwrite_output=True)
        audio_paths[path] = output_path
    return audio_paths

def generate_subtitles(audio_path, srt_path, transcribe):
    print(f"Generating subtitles for {os.path.basename(audio_path)}... This might take a while.")

    warnings.filterwarnings("ignore")
    result = transcribe(audio_path)
    warnings.filterwarnings("default")

    with open(srt_path, "w", encoding="utf-8") as srt:
        write_srt(result["segments"], file=srt)
    return result

def get_subtitles(audio_paths: list, output_srt: bool, output_dir: str, transcribe: callable):
    srt_path = output_dir if output_srt else tempfile.gettempdir()
    subtitles_path = {}

    for path, audio_path in audio_paths.items():
        filename = os.path.basename(path).split('.')[0]
        srt_path = os.path.join(srt_path, f"{filename}.srt")

        result = generate_subtitles(audio_path, srt_path, transcribe)

        subtitles_path[path] = srt_path
    
    return subtitles_path , result

d:\iNeuron\Project_Neuron\video_summarizer\vsum\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Call the subtite functions

In [4]:
def auto_subtitle(video_path:str, model:str, output_dir:str, srt:bool, verbose:bool, 
                ):
    os.makedirs(output_dir, exist_ok=True)
    if model.endswith(".en"):
        print(f"{model} is a English model")
    model = whisper.load_model(model)
    audio = get_audio(video_path)
    subtitle = get_subtitles(
                audio, srt, output_dir, lambda audio_path: model.transcribe(audio_path, 
                                                verbose=verbose, task='transcribe'))
    return subtitle

In [10]:
import sys
vid_path = r"D:\Recordings\final\pyproject\003_Project Setup.mp4"
try:
    subtitle = auto_subtitle(video_path=[vid_path],
    model='tiny.en', output_dir='srt', srt=True, verbose=False)
except Exception as e:
    print(f'error {sys.stderr}')

tiny.en is a English model
Extracting audio from 003_Project Setup.mp4...
Generating subtitles for 003_Project Setup.wav... This might take a while.


100%|██████████| 18421/18421 [00:19<00:00, 965.58frames/s]


In [70]:
subtitle['D:\\iNeuron\\Project_Neuron\\Video summarization\\vid\\sample.mp4'] = 'srt\\sample_mp4.srt'

### Code for adding subtitle to video as overlay

In [95]:
def add_subtitles(path: str, subtitles_path: str, output_dir: str):
    try:
      filename = os.path.basename(path).split('.')[0]
      print(filename)
      out_path = os.path.join(output_dir, f"{filename}.mp4")

      print(f"Adding subtitles to {filename}...")

      video = ffmpeg.input(path)
      audio = video.audio

      stderr = ffmpeg.concat(
        video.filter('subtitles', subtitles_path, force_style="OutlineColour=&H40000000,BorderStyle=3"), audio, v=1, a=1
      ).output(out_path).run(quiet=True, overwrite_output=True)

      print(f"Saved subtitled video to {os.path.abspath(out_path)}.")
    except Exception as e:
        print(e, sys.stderr)

def main(subtitles: dict, output_dir: str):
    try:
      for path, srt_path in subtitles.items():
         add_subtitles(path, srt_path, output_dir)
    except Exception as e:
        print(e, sys.stderr)

main(subtitle[0], r'D:\iNeuron\Project_Neuron\Video summarization')

001_Introduction
Adding subtitles to 001_Introduction...
Saved subtitled video to D:\iNeuron\Project_Neuron\Video summarization\001_Introduction.mp4.


In [84]:
filename = os.path.basename('D:\\iNeuron\\Project_Neuron\\Video summarization\\vid\\sample.mp4').split('.')[0]
print(filename)

sample


### Summarization

In [8]:
from transformers import AutoModelForSeq2SeqLM,AutoTokenizer

# Load tokenizer 
tokenizer = AutoTokenizer.from_pretrained("sshleifer/distilbart-cnn-12-6")
# Load model 
model = AutoModelForSeq2SeqLM.from_pretrained("sshleifer/distilbart-cnn-12-6")

In [11]:
inputs = tokenizer(subtitle[1]['text'], 
                max_length=512, 
                truncation=True,
                return_tensors="pt")

summary_ids = model.generate(inputs["input_ids"], max_new_tokens=512)
tokenizer.batch_decode(summary_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]

" In this video, we will be creating our basic project setup . Let's start with GitHub, then click on Repostries and create a private repository . Next step is we will just clone this report to our local system . We can make whatever changes we want in this local repo and then push it after every change ."